In [2]:
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd




In [3]:
folder = r"C:\Users\jmsch\2025-06-02_wflow_qgis\wflow_test_terracing"
staticmaps = xr.open_dataset(f"{folder}/staticmaps.nc")


terraces_sc2 = gpd.read_file(r"C:\Users\jmsch\2025-06-02_wflow_qgis\wflow_tool_nbs\Terraces_sc2.gpkg")
terraces_sc3 = gpd.read_file(r"C:\Users\jmsch\2025-06-02_wflow_qgis\wflow_tool_nbs\Terraces_sc3.gpkg")


In [4]:
display(staticmaps.Slope)

<xarray.DataArray 'Slope' (latitude: 321, longitude: 233)> Size: 299kB
[74793 values with dtype=float32]
Coordinates:
  * latitude     (latitude) float64 3kB -0.1821 -0.1838 ... -0.7138 -0.7154
  * longitude    (longitude) float64 2kB 35.84 35.84 35.84 ... 36.22 36.22 36.22
    spatial_ref  int32 4B ...
Attributes:
    unit:     m.m-1

In [8]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

# Define the target CRS
target_crs = "EPSG:4326"

# Reproject terraces to target CRS
terraces_sc2 = terraces_sc2.to_crs(target_crs)
terraces_sc3 = terraces_sc3.to_crs(target_crs)

# 1. Get grid cell centers as Points
lon = staticmaps["longitude"].values
lat = staticmaps["latitude"].values

lon_grid, lat_grid = np.meshgrid(lon, lat)
points = [Point(x, y) for x, y in zip(lon_grid.flatten(), lat_grid.flatten())]

points_gdf = gpd.GeoDataFrame(geometry=points, crs=target_crs)

# Reproject to UTM for accurate 50 m buffer
utm_crs = "EPSG:32736"  # Adjust to your local UTM zone
points_gdf_utm = points_gdf.to_crs(utm_crs)
points_gdf_utm["geometry"] = points_gdf_utm.buffer(50)

# Reproject terraces to UTM for intersection
terraces_sc2_utm = terraces_sc2.to_crs(utm_crs)
terraces_sc3_utm = terraces_sc3.to_crs(utm_crs)

# 2. Determine which points intersect terraces_sc2 and terraces_sc3 separately
mask_sc2 = points_gdf_utm.intersects(terraces_sc2_utm.unary_union)
mask_sc3 = points_gdf_utm.intersects(terraces_sc3_utm.unary_union)

# 3. Update slope values separately
slope = staticmaps["Slope"].values.flatten()

# Save original slope values for comparison
points_gdf_utm["orig_slope"] = slope

# Apply 0.5 reduction for terraces_sc2
slope[mask_sc2] *= 0.5
points_gdf_utm["slope_sc2"] = np.where(mask_sc2, slope, np.nan)

# Apply 0.5 reduction for terraces_sc3
slope[mask_sc3] *= 0.5
points_gdf_utm["slope_sc3"] = np.where(mask_sc3, slope, np.nan)

# 4. Reshape back to grid
slope_reshaped = slope.reshape(lon_grid.shape)
staticmaps["Slope"].values[:] = slope_reshaped

# Save the modified staticmaps
staticmaps.to_netcdf(f"{folder}/staticmaps_modified.nc")

# 5. Reproject back to EPSG:4326 for display
points_gdf_final = points_gdf_utm.to_crs(target_crs)

# 6. Display only points where slope was adjusted
adjusted_sc2 = points_gdf_final.loc[mask_sc2, ["geometry", "orig_slope", "slope_sc2"]]
adjusted_sc3 = points_gdf_final.loc[mask_sc3, ["geometry", "orig_slope", "slope_sc3"]]

print("Slope-adjusted cells within terraces_sc2:")
print(adjusted_sc2)

print("\nSlope-adjusted cells within terraces_sc3:")
print(adjusted_sc3)


C:\Users\jmsch\AppData\Local\Temp\ipykernel_25320\968684940.py:31: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  mask_sc2 = points_gdf_utm.intersects(terraces_sc2_utm.unary_union)
C:\Users\jmsch\AppData\Local\Temp\ipykernel_25320\968684940.py:32: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  mask_sc3 = points_gdf_utm.intersects(terraces_sc3_utm.unary_union)


Slope-adjusted cells within terraces_sc2:
                                                geometry  orig_slope  \
7093   POLYGON ((36.00837 -0.23208, 36.00836 -0.23213...    0.010224   
7327   POLYGON ((36.01003 -0.23375, 36.01003 -0.23379...    0.011097   
7559   POLYGON ((36.00837 -0.23542, 36.00836 -0.23546...    0.018162   
7561   POLYGON ((36.0117 -0.23542, 36.0117 -0.23546, ...    0.023014   
7562   POLYGON ((36.01337 -0.23542, 36.01336 -0.23546...    0.035345   
...                                                  ...         ...   
15733  POLYGON ((36.04003 -0.29375, 36.04003 -0.29379...    0.041123   
15963  POLYGON ((36.03503 -0.29542, 36.03503 -0.29546...    0.026516   
15965  POLYGON ((36.03837 -0.29542, 36.03836 -0.29546...    0.030114   
16197  POLYGON ((36.0367 -0.29708, 36.0367 -0.29713, ...    0.013095   
16430  POLYGON ((36.0367 -0.29875, 36.0367 -0.29879, ...    0.006566   

       slope_sc2  
7093    0.005112  
7327    0.005548  
7559    0.009081  
7561    0.011507 